# Instructions
Run the first couple of cells.

When you get to the generated Meshcat URL, click to open it. Read the notes below the cell to set up your gamepad or see/set the velocity command values mapped to your keyboard.
**note:** driving with the gamepad is a much better experience!!

Then, run the final cell and drive the robot around! Observe the many realistic forces acting upon the robot (contact forces, gravity, inertia, etc). The only way that the robot is moving is because the motors apply a torque at the wheel and Drake simulates the detailed physics of the mecanum wheel rollers.

In [ ]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    DiscreteContactApproximation,
    LeafSystem,
    MeshcatVisualizer,
    Parser,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatSliders, StopButton
from manipulation.utils import ConfigureParser, FindResource

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

If you want to use a gamepad to steer, press some buttons on the gamepad now with the meshcat window in focus.

You can control the vehicle with either a gamepad (if you have it) or with a keyboard teleop (if you don't). If you're using the keyboard, then click "Open Controls" in meshcat to see/set the velocity command values. The sliders are also mapped to the keys:
```
xdot : [ArrowDown, ArrowUp]
ydot : [ArrowLeft, ArrowRight]
yawdot : [KeyA, KeyD]
```


In [ ]:
class GamepadCommand(LeafSystem):
    def __init__(self, meshcat):
        LeafSystem.__init__(self)

        self._meshcat = meshcat
        port = self.DeclareVectorOutputPort("command", 3, self.CalcOutput)
        port.disable_caching_by_default()

    def CalcOutput(self, context, output):
        gamepad = self._meshcat.GetGamepad()
        assert gamepad.index != None

        # https://beej.us/blog/data/javascript-gamepad/
        def CreateStickDeadzone(x, y):
            stick = np.array([x, y])
            deadzone = 0.2
            m = np.linalg.norm(stick)
            if m < deadzone:
                return np.array([0, 0])
            over = (m - deadzone) / (1 - deadzone)
            return stick * over / m

        left = CreateStickDeadzone(gamepad.axes[0], gamepad.axes[1])
        right = CreateStickDeadzone(gamepad.axes[2], gamepad.axes[3])

        output[0] = -left[1]  # Left stick y => vx
        output[1] = -left[0]  # Left stick x => vy
        output[2] = right[0]  # Right stick x => wz


class MyController(LeafSystem):
    def __init__(self, plant, model_instance):
        LeafSystem.__init__(self)
        self._wheel_velocity_indices = (
            np.array(
                [
                    plant.GetJointByName(
                        "front_left_wheel_joint", model_instance
                    ).velocity_start(),
                    plant.GetJointByName(
                        "front_right_wheel_joint", model_instance
                    ).velocity_start(),
                    plant.GetJointByName(
                        "rear_left_wheel_joint", model_instance
                    ).velocity_start(),
                    plant.GetJointByName(
                        "rear_right_wheel_joint", model_instance
                    ).velocity_start(),
                ]
            )
            + plant.num_positions()
        )

        # command is the [vx, vy, wz] components of V_WRobot_Robot.
        self.DeclareVectorInputPort("command", 3)
        self.DeclareVectorInputPort("state", plant.num_multibody_states())
        self.DeclareVectorOutputPort("motor_torque", 4, self.CalcTorques)

        # These should match the parameters used to create the URDF.
        wheel_radius = 0.045 + (0.015 / 2)  # hub_radius + (roller_diameter / 2).
        wheelbase = 0.2
        track = 0.205
        self._wheel_velocity_kp = 0.1

        lx = track * 0.5
        ly = wheelbase * 0.5

        # From anzu/punito/control/mecanum_kinematics.cc
        self._vehicle_to_wheel_map = (
            np.array(
                [
                    [1, -1, (lx + ly)],
                    [1, 1, -(lx + ly)],
                    [1, 1, (lx + ly)],
                    [1, -1, -(lx + ly)],
                ]
            )
            / wheel_radius
        )

    def CalcTorques(self, context, output):
        # From anzu/punito/sim/robot_master_controller.cc
        command = self.get_input_port(0).Eval(context)
        state = self.get_input_port(1).Eval(context)
        wheel_velocity = state[self._wheel_velocity_indices]

        desired_wheel_velocity = self._vehicle_to_wheel_map @ command
        torque = self._wheel_velocity_kp * (desired_wheel_velocity - wheel_velocity)

        output.SetFromVector(torque)


def teleop():
    builder = DiagramBuilder()

    time_step = 0.005
    plant, scenegraph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModels(FindResource("models/mecanum/drake_obstacles.dmd.yaml"))
    plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
    plant.Finalize()
    robot_instance = plant.GetModelInstanceByName("mecanum_base")

    controller = builder.AddSystem(MyController(plant, robot_instance))
    builder.Connect(plant.get_state_output_port(), controller.get_input_port(1))
    builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    meshcat.SetProperty("/Lights/PointLightNegativeX", "visible", False)

    gamepad = meshcat.GetGamepad()
    if gamepad.index != None:  # use gamepad teleop
        print("Gamepad found.  Using gamepad teleop.")
        gamepad_command = builder.AddSystem(GamepadCommand(meshcat))
        builder.Connect(gamepad_command.get_output_port(), controller.get_input_port(0))

    else:  # use keyboard teleop
        print("No gamepad found.  Using keyboard/slider teleop.")
        step = 0.02
        meshcat.AddSlider(
            name="xdot",
            increment_keycode="ArrowUp",
            decrement_keycode="ArrowDown",
            min=-3,
            max=3,
            step=step,
            value=0,
        )
        meshcat.AddSlider(
            name="ydot",
            increment_keycode="ArrowLeft",
            decrement_keycode="ArrowRight",
            min=-1,
            max=1,
            step=step,
            value=0,
        )
        meshcat.AddSlider(
            name="yawdot",
            increment_keycode="KeyD",
            decrement_keycode="KeyA",
            min=-1,
            max=1,
            step=step,
            value=0,
        )
        sliders = builder.AddSystem(
            MeshcatSliders(meshcat, [["xdot", "ydot", "yawdot"]])
        )
        builder.Connect(sliders.get_output_port(), controller.get_input_port(0))

    # Add a meshcat visualizer.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scenegraph.get_query_output_port(), meshcat
    )
    builder.AddSystem(StopButton(meshcat))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_context()

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)
        simulator.AdvanceTo(np.inf)
    else:
        simulator.AdvanceTo(0.1)


teleop()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=57b741a1-558d-4f78-92d3-c473c39676c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>